In [1]:
from IPython.display import Image

# Introduction to Pywr

## Pywr is a generalised network resource allocation model written in Python. 
## It aims to be fast, free, and extendable.


### by James Tomlinson

# Overview

This presentation covers:

- A quick background to Pywr 
- A simple example model using the JSON format
- An overview of how to extend Pywr 

# Background to Pywr

- Pywr is a tool for **solving network resource allocation problems** at discrete timesteps using a linear programming approach. 
- It's principal application is in resource allocation in water supply networks, although other uses are conceivable.
- Nodes in the network can be given constraints (e.g. minimum/maximum flows) and costs, and can be connected as required. 
- Parameters in the model can vary time according to boundary conditions (e.g. an inflow timeseries) or based on states in the model (e.g. the current volume of a reservoir).
- Models can be developed using the Python API, either in a script or interactively using IPython/Jupyter. 
- Alternatively, models can be defined in a rich JSON-based document format.



# Networks networks networks

![](pywr_d3.png)

# Design goals

- Pywr is a tool for solving network resource allocation problems. 
     - Some similarities with other software packages such as WEAP, Wathnet and Aquator, but also has some significant differences.
- Pywr’s principle design goals are that it is:
     - Fast enough to handle large datasets, and large numbers of scenarios and function evaluations required by advanced decision making methodologies;
     - Free to use without restriction – licenced under the GNU General Public Licence;
     - Extendable – using the Python programming language to define complex operational rules and control model runs


# Conceptual Pywr model run

The following is a pseudo-code conceptualisation of a Pywr model run. The actual code is a little (but not much!) more complicated.

```python
model = Model.load('mymodel.json')  # Load a model from a JSON definition.
model.setup()  # Do some initial setup.
for timestep in model.timesteps:
    model.before()  # Update Nodes etc. before the solve this time-step.
    model.solve()   # Allocate the resource around the network.
    model.after()   # Update Nodes etc. after the solve this time-step.
model.finish()  # Finalise anything (e.g. close files).
```

# Technical overview

- Native support for multiple scenarios. 
    - Not parallel execution but multiple scenarios can be run during a single simulation.
    - Easy to define which data vary in which scenarios (most of your data does not vary!).
    - Scenarios can be sliced for different runs and therefore can exploit batch runnning (multiple processors).
- Resource agnostic.
    - Primarily used for water networks.
    - However, any resource flow could be modelled. Including multi-resource models.
- JSON input file format.
    - Models can be defined using a JSON format. 
    - This allows models to be shared (e.g. over the web), templated and manipulated by existing JSON tools.

# Technical overview (2)

- Native support for multiple scenarios. 
    - Not parallel execution but multiple scenarios can be run during a single simulation.
    - Easy to define which data vary in which scenarios (most of your data does not vary!).
    - Scenarios can be sliced for different runs and therefore can exploit batch runnning (multiple processors).
- Resource agnostic.
    - Primarily used for water networks.
    - However, any resource flow could be modelled. Including multi-resource models.
- JSON input file format.
    - Models can be defined using a JSON format. 
    - This allows models to be shared (e.g. over the web), templated and manipulated by existing JSON tools.

# The most basic example

Let's work through the most basic Pywr example. 

This example uses the Python API (as opposed to JSON input file).

First import the required classes from Pywr ...

In [2]:
from pywr.core import Model
from pywr.nodes import Input, Output, Link
from pywr.notebook import draw_graph


... create a new `Model` instance ...

In [3]:
model = Model()

... add some nodes ...

In [4]:
input_node = Input(model, name='Input')
link_node = Link(model, name='Link')
output_node = Output(model, name='Output')

... connect them together ....

In [5]:
input_node.connect(link_node)
link_node.connect(output_node)

... and finally draw a representation of the network.

In [6]:
draw_graph(model, height=100)

<IPython.core.display.Javascript object>

In [7]:
model.run();

output_node.flow

array([ 0.])

Update the input flow

    - Maximum flow of 10 $Mm^3$/day.
    - Negative cost induces a flow through the only route in the model.

In [8]:
input_node.max_flow = 10
input_node.cost = -1

model.run();
output_node.flow

array([ 10.])